In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import pandas as pd
import time
import os
import shutil

In [3]:
def getHTML(url):
    driver = webdriver.Chrome(service =Service(ChromeDriverManager().install()))
    driver.get(url)
    with open("gdpPage.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)

In [4]:
# url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita"
# getHTML(url)

In [5]:
pageFile =  open("gdpPage.html", encoding="utf-8")
gdpPage = pageFile.read()
soup = BeautifulSoup(gdpPage,'html.parser')

In [6]:
print(soup.title)

<title>List of countries by GDP (nominal) per capita - Wikipedia</title>


In [7]:
table = soup.find_all("table")
for t in table:
    print(t.get("class"))

None
['wikitable', 'sortable', 'static-row-numbers', 'plainrowheaders', 'srn-white-background']
['nowraplinks', 'mw-collapsible', 'uncollapsed', 'navbox-inner']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'autocollapse', 'navbox-inner']


In [8]:
def getTable(htmlFile):
    driver = webdriver.Chrome(service =Service(ChromeDriverManager().install()))
    driver.get(htmlFile)
    gdpTable = driver.find_element(By.XPATH, "//*[@id='mw-content-text']/div[1]/table")
    print(driver.page_source)   

In [26]:
def cleanCountryName(text):
    return text[:len(text) - 2]

In [28]:
def putInEstimate(curRow, infoDictionary, agency, colIndex):
    """
        Put GDP estimate and year from the agency into the corresponding info dictionary.
        If there was no estimate for that agency, insert (0, 0).
        Arguments:
            curRow: The current <tr> element
            infoDictionary: dictionary to put the information into
            agency: "IMF", "UN", or "World Bank"
            colIndex: the current column
        return the column index - will be colIndex + 1 if no estimate, +2 if there was an estimate
    """
    keyToPutInfo = agency + " Estimate"
    
    if curRow[colIndex].has_attr('colspan'):
        infoDictionary[keyToPutInfo] = 0
        infoDictionary[agency + " Year"] = 0
        return colIndex + 1
    else:
        infoDictionary[keyToPutInfo] = curRow[colIndex].text
        infoDictionary[agency + " Year"] = curRow[colIndex + 1].text.split()[0]
        return colIndex + 2

In [29]:
gdpTable = soup.find("table", class_="wikitable sortable static-row-numbers plainrowheaders srn-white-background")
gdpTableData = gdpTable.find("tbody")
gdpRows = gdpTableData.find_all("tr")
columns = ["Country/Territory", "UN Region", "IMF Estimate", " IMF Year", "UN Estimate", "UN Year",
           "World Bank Estimate", " World Bank Year" ]
df = pd.DataFrame(columns=columns)

pdRow = 0;

for i in range(2,len(gdpRows)):
    infoDictionary = {}
    
    r = gdpRows[i].find_all(["td"])
    if (len(r) != 0):
        infoDictionary['Country/Territory'] = cleanCountryName(r[0].find("a").text)
        infoDictionary['UN Region'] = r[1].text
        colIndex = 2
        agencies = ["IMF", "UN", "World Bank"]
        for agency in agencies:
            colIndex = putInEstimate(r, infoDictionary, agency, colIndex)
        print(infoDictionary)
        
    
        
        
        
        


{'Country/Territory': 'Liechtenstein', 'UN Region': 'Europe', 'IMF Estimate': 0, 'IMF Year': 0, 'UN Estimate': '180,227', 'UN Year': '2020', 'World Bank Estimate': '175,814', 'World Bank Year': '2019'}
{'Country/Territory': 'Monaco', 'UN Region': 'Europe', 'IMF Estimate': 0, 'IMF Year': 0, 'UN Estimate': '173,696', 'UN Year': '2020', 'World Bank Estimate': '190,513', 'World Bank Year': '2019'}
{'Country/Territory': 'Luxembourg', 'UN Region': 'Europe', 'IMF Estimate': '135,046', 'IMF Year': '2022', 'UN Estimate': '117,182', 'UN Year': '2020', 'World Bank Estimate': '115,874', 'World Bank Year': '2020'}
{'Country/Territory': 'Bermuda', 'UN Region': 'Americas', 'IMF Estimate': 0, 'IMF Year': 0, 'UN Estimate': '123,945', 'UN Year': '2020', 'World Bank Estimate': '107,080', 'World Bank Year': '2020'}
{'Country/Territory': 'Ireland', 'UN Region': 'Europe', 'IMF Estimate': '101,509', 'IMF Year': '2022', 'UN Estimate': '86,251', 'UN Year': '2020', 'World Bank Estimate': '85,268', 'World Bank Y